In [2]:
import findspark
findspark.init('/home/mukund/spark-2.1.0-bin-hadoop2.7/')
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('miss').getOrCreate()

In [3]:
df=spark.read.csv('/home/mukund/as/ContainsNull.csv',inferSchema=True,header=True)

In [4]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [5]:
df.na.drop().show() #simply dropping all the null values

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [10]:
a=df.na.drop(thresh=2).show() #drop all rows which have 2 or more than 2 null

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [14]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [16]:
df.na.drop(how='any').show() #any null value, then just drop

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [19]:
df.na.drop(how='all').show() #nothing gets dropped as none of the rows have null

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [21]:
df.na.drop(subset=['sales']).show() #drop rows for which sales are null

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [22]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [23]:
df.na.fill('FillEmpty').show() #spark automatically fills in the value passed to only string type columns

+----+---------+-----+
|  Id|     Name|Sales|
+----+---------+-----+
|emp1|     John| null|
|emp2|FillEmpty| null|
|emp3|FillEmpty|345.0|
|emp4|    Cindy|456.0|
+----+---------+-----+



In [27]:
#however to avoid spark filling all string type columns with this value,use subset
df.na.fill('FillEmpty',subset=['Name']).show()

+----+---------+-----+
|  Id|     Name|Sales|
+----+---------+-----+
|emp1|     John| null|
|emp2|FillEmpty| null|
|emp3|FillEmpty|345.0|
|emp4|    Cindy|456.0|
+----+---------+-----+



In [28]:
df.na.fill(10,subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| 10.0|
|emp2| null| 10.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [35]:
from pyspark.sql.functions import mean #filling missing data with mean

In [36]:
mv=df.select(mean(df['sales'])).collect()

In [37]:
ms=mv[0][0]

In [38]:
df.na.fill(ms,subset=['sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

